**Librerías**

In [12]:
import pandas as pd
import re

**Declaración de Función Limpieza de SMS**

In [ ]:
def clean_sms_v2(file_path, output_path):
    '''Función para limpiar bases de SMS para validar 
    que el número sea igual a 10 dígitos y los nombres 
    de los clientes no contengan carácteres especiales.'''

    # Leer archivo
    print("FUNCIÓN PARA LIMPIEZA DE BASES DE SMS")
    df = pd.read_csv(file_path)
    print(f"Tu DataFrame tiene: {len(df)} registros antes de la limpieza")
    print()

    # Limpiar el campo NOMBRES (primer nombre, sin caracteres especiales ni números)
    df['NOMBRES'] = (
        df['NOMBRES']
        .str.split()
        .str[0]
        .str.replace(r'[^A-Za-z]+', "" , regex=True)
    )
    
    # Remplazos de palabras particulares
    df['NOMBRES'] = df['NOMBRES'].replace('MA', 'MARIA')
    df['NOMBRES'] = df['NOMBRES'].replace('A', 'ESTIMADO')

    # Sección de nulos
    print("SECCIÓN DE NULOS")
    print("Número de registros nulos eliminados en 'TELEFONO':", df['TELEFONO'].isna().sum())
    df.replace("", np.nan, inplace=True)
    df = df[df['TELEFONO'].notna()].reset_index(drop=True)
    print("Número de registros llenados con 'ESTIMADO' en 'NOMBRES':", df['NOMBRES'].isna().sum())
    df['NOMBRES'] = df['NOMBRES'].fillna('ESTIMADO')
    print()
    
    #Conversión de datos
    df['NOMBRES'] = df['NOMBRES'].astype(str)
    df['TELEFONO'] = df['TELEFONO'].astype(float)
    df['TELEFONO'] = df['TELEFONO'].astype(int)
    df['TELEFONO'] = df['TELEFONO'].astype(str)

    # Eliminar múltiples números celulares (solo conservar el primero)
    df['TELEFONO'] = df['TELEFONO'].str.split().str[0]

    # Calcular registros con longitud diferente a 10
    registros_invalidos_10 = len(df[df['TELEFONO'].str.len() != 10])

    # Filtrar registros con números de teléfono válidos (10 dígitos y no seriados)
    invalid_numbers = ['5555555555', '0000000000', '1234567890']
    registros_invalidos_lista = len(df[df['TELEFONO'].isin(invalid_numbers)])

    df = df[
        (df['TELEFONO'].str.len() == 10) &
        (~df['TELEFONO'].isin(invalid_numbers))
    ]
    
    print ("SECCIÓN DE NÚMEROS ERRÓNEOS")
    print(f"Registros eliminados por longitud de teléfono diferente a diez: {registros_invalidos_10}")
    print(f"Registros eliminados por coincidir con números inválidos: {registros_invalidos_lista}")
    print()

    # Exportación  de DataFrame limpio
    print(f"Tu DataFrame tiene: {len(df)} registros despúes de la limpieza")
    
    df.to_csv(output_path, index=False)

    return "Felicidades has limpiado tu base de sms"



**Aplicación**

In [14]:
# Ejemplo de uso
clean_sms(r"C:\Users\lgonzalezc\Downloads\prueba_sms.csv",
          r"C:\Users\lgonzalezc\Downloads\prueba_sms_limpia.csv"
          )


'Felicidades, has limpiado tu base de SMS'